## ATIVIDADE 2 - Web Scraping no site IMDB

In [112]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import selenium
import time
selenium.__version__

'4.31.1'

In [113]:
driver = webdriver.Chrome()

In [114]:
driver.get("https://www.imdb.com")

In [116]:
menu = driver.find_element(By.XPATH,r'//*[@id="imdbHeader-navDrawerOpen"]') 

In [117]:
print(type(menu))

<class 'selenium.webdriver.remote.webelement.WebElement'>


In [118]:
menu.click()

In [119]:
link_calendar = driver.find_element(By.LINK_TEXT, r'Calendário de lançamento')

In [120]:
print(type(link_calendar))
print(link_calendar.tag_name)

<class 'selenium.webdriver.remote.webelement.WebElement'>
a


In [121]:
url = link_calendar.get_attribute("href")
print(url)

https://www.imdb.com/calendar/?ref_=nv_mv_cal


In [122]:
driver.get(url)

In [123]:
tag_ul = driver.find_elements(By.CLASS_NAME, 'ipc-metadata-list-summary-item')


In [124]:
print(type(tag_ul))

<class 'list'>


In [125]:
for movie in tag_ul:
    print(movie.find_element(By.CLASS_NAME, 'ipc-metadata-list-summary-item__t').text)

Longlegs: Vínculo Mortal (2024)
Afraid (2024)
Sting: Aranha Assassina (2024)
A Caça (2024)
Estômago II: O Poderoso Chef (2024)
Cidade; Campo (2024)
Os Fantasmas Ainda se Divertem - Beetlejuice Beetlejuice (2024)
Kill - O Massacre no Trem (2023)
O Menino e o Mestre (2023)
Corrida Maluca (2023)
Vovó Ninja (2024)
Não Fale o Mal (2024)
Transformers: O Início (2024)
Meu Amigo Pinguim (2024)
O Bastardo (2023)
Ligação Sombria (2023)
Silvio (2024)
Un silence (2023)
Meu Casulo de Drywall (2023)
A Forja: O Poder da Transformação (2024)
Hellboy e o Homem Torto (2024)
Golpe de Sorte em Paris (2023)
Passagrana (2024)
Saudade fez Morada aqui Dentro (2022)
Robô Selvagem (2024)
Coringa: Delírio a Dois (2024)
Não Solte! (2024)
Infestação (2023)
A Garota da Vez (2023)
Pássaro Branco: Uma História de Extraordinário (2023)
A Menina e o Dragão (2024)
Força Bruta: Sem Saída (2023)
Tudo Por Um Pop Star 2 (2024)
Sorria 2 (2024)
O Aprendiz (2024)
Super/Man: A História de Christopher Reeve (2024)
Perfekta: Uma 

In [126]:
def extract_titulos(movie):
    title_element = movie.find_element(By.CSS_SELECTOR, 'a.ipc-metadata-list-summary-item__t')
    return title_element.text

def extract_data_lancamento(section):
    return section.find_element(By.CSS_SELECTOR, 'h3.ipc-title__text').text

def extract_generos(movie):
    genres = movie.find_elements(By.CSS_SELECTOR, 'ul.ipc-inline-list--show-dividers.ipc-metadata-list-summary-item__tl li')
    return [genre.text for genre in genres]

def extract_links(movie, lista):
    title_element = movie.find_element(By.CSS_SELECTOR, 'a.ipc-metadata-list-summary-item__t')
    link = title_element.get_attribute('href')
    lista.append(link)
    return link



In [127]:
def extract_movies_info():
    movies_list = []
    global movies_links
    movies_links = []

    # Encontrar todas as seções de lançamento
    sections = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="calendar-section"]')
    
    for section in sections:
        # Extrair a data da seção
        date = extract_data_lancamento(section)
        
        # Encontrar todos os filmes na seção
        movies = section.find_elements(By.CSS_SELECTOR, 'li[data-testid="coming-soon-entry"]')
        
        for movie in movies:
            movie_info = {}
            movie_info['Título'] = extract_titulos(movie)
            movie_info['Link'] = extract_links(movie, movies_links)
            movie_info['Gêneros'] = extract_generos(movie)
            movie_info['Data de lançamento'] = date
            
            movies_list.append(movie_info)
    
    return movies_list

In [129]:
import pandas as pd

# Supondo que você já executou a função extract_movies_info() e obteve a lista de filmes
movies_list = extract_movies_info()

# Criar um DataFrame a partir da lista de dicionários
df_movies = pd.DataFrame(movies_list)

# Exibir o DataFrame
display(df_movies)


,Título,Link,Gêneros,Data de lançamento
0,Longlegs: Vínculo Mortal (2024),https://www.imdb.com/title/tt23468450/?ref_=rlm,"[Policial, Terror, Suspense]",29 de ago. de 2024
1,Afraid (2024),https://www.imdb.com/title/tt24577462/?ref_=rlm,"[Terror, Mistério, Ficção científica]",29 de ago. de 2024
2,Sting: Aranha Assassina (2024),https://www.imdb.com/title/tt20112746/?ref_=rlm,"[Terror, Ficção científica, Suspense]",29 de ago. de 2024
3,A Caça (2024),https://www.imdb.com/title/tt22493538/?ref_=rlm,"[Terror, Suspense]",29 de ago. de 2024
4,Estômago II: O Poderoso Chef (2024),https://www.imdb.com/title/tt28281591/?ref_=rlm,"[Comédia, Policial, Drama]",29 de ago. de 2024
...,...,...,...,...
77,Thunderbolts* (2025),https://www.imdb.com/title/tt20969586/?ref_=rlm,"[Ação, Aventura, Policial]",02 de mai. de 2025
78,Missão: Impossível 8 (2025),https://www.imdb.com/title/tt9603208/?ref_=rlm,"[Ação, Aventura, Suspense]",22 de mai. de 2025
79,Karate Kid (2025),https://www.imdb.com/title/tt1674782/?ref_=rlm,"[Ação, Drama, Família]",30 de mai. de 2025
80,F1 (2025),https://www.imdb.com/title/tt16311594/?ref_=rlm,"[Drama, Esportes]",26 de jun. de 2025


In [132]:
def procura_ator_diretor(aux, link):
    driver.get(link)

    director_element = driver.find_element(By.CSS_SELECTOR, '.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    

    actor_elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-testid="title-cast-item__actor"]')
    actors = [actor.text for actor in actor_elements]

    aux.append(director_element.text)
    aux.append(actors)

    return aux

In [133]:
info = []
for link in movies_links:
    aux = []
    info.append(procura_ator_diretor(aux, link))

for i in info:
    print(i)

['Osgood Perkins', ['Maika Monroe', 'Nicolas Cage', 'Blair Underwood', 'Alicia Witt', 'Michelle Choi-Lee', 'Dakota Daulby', 'Lauren Acala', 'Kiernan Shipka', 'Maila Hosie', 'Jason William Day', 'Lisa Chandler', 'Ava Kelders', 'Rryla McIntosh', 'Carmel Amit', 'Shafin Karim', 'Trey Helten', 'Daniel Bacon', 'Vanessa Walsh']]
['Chris Weitz', ['John Cho', 'Katherine Waterston', 'Keith Carradine', 'Havana Rose Liu', 'Lukita Maxwell', 'Ashley Romans', 'David Dastmalchian', 'Wyatt Lindner', 'Isaac Bae', 'Bennett Curran', 'Greg Hill', 'Riki Lindhome', 'Ashton Essex Bright', 'Mason Shea Joyce', 'River Drosche', 'Todd Waring', 'Simon Craig Raynes', 'Rogelio Douglas III']]
['Kiah Roache-Turner', ['Noni Hazlehurst', 'Jermaine Fowler', 'Alyla Browne', 'Robyn Nevin', 'Ryan Corr', 'Kate Walsh', 'Penelope Mitchell', 'Jett Berry', 'Kade Berry', 'Silvia Colloca', 'Lee Perry', 'Danny Kim', 'Tony J Black', 'Alcira Carpio', 'Rowland Holmes']]
['Louis Lagayette', ['Lily Banda', 'Alec Newman', 'Kostis Kallivr